# Imports

In [77]:
import json
from operator import itemgetter
from typing import List, Literal, Optional, Union

import chromadb
from dotenv import load_dotenv
from IPython.display import Markdown
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnableLambda, Runnable
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI
from langsmith import traceable
from pydantic import BaseModel, Field

# Initialization

In [78]:
load_dotenv()  # For MISTRAL_API_KEY and LangSmith tracing

True

In [79]:
settings = {
    "llm": {"model_name": "mistral-large-latest"},
    "emb_model": {"model_name": "deepvk/USER-bge-m3"},
}

In [80]:
# Load models
llm = ChatMistralAI(**settings["llm"])
emb_model = HuggingFaceEmbeddings(**settings["emb_model"])

# Query classification and analysis

In [ ]:
with open("prompts/system_prompt_1.txt") as f:
    system_prompt_1 = f.read()

with open("prompts/examples_1.json") as f:
    examples_1 = json.load(f)

example_prompt_1 = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}"),
        ("assistant", "{output}"),
    ]
)

few_shot_prompt_1 = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt_1,
    examples=examples_1,
)

prompt_1 = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_1),
        few_shot_prompt_1,
        ("user", "{question}"),
    ]
)

In [82]:
class RAGQueries(BaseModel):
    """
    Данные, сформированные на основе запроса пользователя для поиска релевантных документов по юридической практике в векторной БД
    Необходимы для уточнения запроса и повышения релевантности выдачи
    """

    keyinfo: str = Field(description="Ключевая информация")
    rephrase: List[str] = Field(
        description="Список из двух различных запросов в векторную базу данных, содержащую судебные дела",
    )
    codex: Optional[
        Literal["Гражданский кодекс", "Уголовный кодекс", "Административный кодекс"]
    ] = Field(
        default=None,
        description="Фильтр для уточнения правового поля запроса, используемый при обращении к векторной базе данных, если возможно определить",
    )


class ChitChatResponse(BaseModel):
    """
    Ответ на запрос пользователя, который не предполагает юридической консультации.
    """

    response: str = Field(..., description="Ответ на общий или неюридический запрос пользователя.")


class FirstResponse(BaseModel):
    """
    Итоговый результат обработки запроса пользователя.
    В зависимости от характера запроса результат может быть представлен в двух форматах:
    - RAGQueries: используется для юридических запросов. Содержит структурированные данные для обращения к векторной базе с целью поиска релевантных документов.
    - ChitChatResponse: применяется для общих или неюридических запросов. Содержит текстовый ответ на запрос пользователя.
    """

    response: Union[RAGQueries, ChitChatResponse] = Field(
        ...,
        description="""Итоговый результат обработки запроса пользователя. 
    Форматы ответа:
    - RAGQueries: для юридических запросов, содержит структурированные данные для поиска документов в векторной базе.
    - ChitChatResponse: для общих запросов, содержит текстовый ответ.""",
    )


In [83]:
llm_1 = llm.with_structured_output(FirstResponse)
chain_1 = prompt_1 | llm_1

# Retrieval and final generation

In [84]:
@traceable
def create_queries(inputs: dict) -> dict:
    """
    Creates 3 queries for retriever:
        1. Original user query
        2. Key info + query rephrase 1
        3. Key info + query rephrase 2

    Params:
        `inputs` - output from previous LangChain Runnable in chain
    """
    original_query: str = inputs["question"]
    llm_processed: RAGQueries = inputs["response_1"].response
    
    queries = [original_query]
    for s in llm_processed.rephrase:
        queries.append(llm_processed.keyinfo + "\n" + s)
    return {"queries": queries, "codex_filter": llm_processed.codex}

In [85]:
# Set up ChromaDB with legal practices docs
# Note: this DB should already be initialized
persistent_client = chromadb.PersistentClient(path="../legal_practice_db/vector_storage")
legal_practices_store = Chroma(
    client=persistent_client,
    collection_name="legal_practices",
    embedding_function=emb_model,
)
# Create retriever
# Note: k = 3, because we pick the most relevant `unique` doc for each query (we have 3 queries)
legal_practices_retriever = legal_practices_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [86]:
@traceable
def batch_query(inputs: dict) -> dict:
    """
    Makes batch invoke of retriever for each query. 
    Also set the filter for codex (if it's not None).

    Params:
        `inputs` - output from previous LangChain Runnable in chain
    """
    # Add/remove codex filter for retriever
    codex_poss_values = ["А", "АГ", "АУ", "АГУ", "Г", "ГУ", "У"]
    if inputs["codex_filter"] and inputs["codex_filter"][0] in "АГУ":
        legal_practices_retriever.search_kwargs["filter"] = {
            "codex": {"$in": [val for val in codex_poss_values if inputs["codex_filter"][0] in val]}
        }
    else:
        legal_practices_retriever.search_kwargs.pop("filter", None)

    # Batch retrieve
    relevant_docs = legal_practices_retriever.batch(inputs["queries"])

    return {"relevant_docs": relevant_docs, "orig_query": inputs["queries"][0]}

In [87]:
@traceable
def prepare_docs(inputs) -> dict:
    """
    Post-processing of retrieved relevant docs.
    Steps:
        1. Picks up the most relevant docs for each query w/o duplicates
        2. Add `theme` from metadata of doc to its content
        3. Merges all docs in one string

    Params:
        `inputs` - output from previous LangChain Runnable in chain
    """
    relevant_docs = inputs["relevant_docs"]
    # Pick top document for each query without duplicates
    seen_uids = set()
    top_docs = []
    for query_docs in relevant_docs:  # relevant_docs is a list of lists (docs per query)
        for doc in query_docs:
            uid: str = doc.metadata.get("uid")
            if uid not in seen_uids:
                seen_uids.add(uid)
                top_docs.append(doc)
                break

    # Merge `theme` into `page_content`
    for doc in top_docs:
        theme = doc.metadata.get("theme", "")
        doc.page_content = f"{theme}\n{doc.page_content}"

    # Merge all docs
    context = "\n\n".join(doc.page_content for doc in top_docs)

    return {"context": context, "orig_query": inputs["orig_query"]}

In [ ]:
with open("prompts/system_prompt_2.txt") as f:
    system_prompt_2 = f.read()

prompt_2 = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_2),
        ("user", "{orig_query}\n===\n{context}"),
    ]
)

In [89]:
class ResponseWithLegalDocs(BaseModel):
    """
    Модель для структурированного ответа с юридическими документами.
    """

    text_response: str = Field(
        description="Полный текст юридического ответа на запрос пользователя."
    )
    legal_docs: List[str] = Field(
        description="Список названий юридических документов, использованных для формирования ответа.",
    )

In [90]:
llm_2 = llm.with_structured_output(ResponseWithLegalDocs)

chain_2 = (
    RunnableLambda(create_queries)
    | RunnableLambda(batch_query)
    | RunnableLambda(prepare_docs)
    | prompt_2
    | llm_2
)

# Final chain

In [91]:
@traceable
def route(inputs) -> str | Runnable:
    """
    Swithes behavior of pipeline based on query classification:
        1. If LLM classify query as chit-chat - we just return the text response without doing RAG
        2. Otherwise, if query belongs to legal field - execute `chain_2` for RAG

    Params:
        `inputs` - output from previous LangChain Runnable in chain
    """
    if isinstance(inputs["response_1"].response, ChitChatResponse):
        return inputs["response_1"].response.response
    else:
        return chain_2

In [92]:
final_chain = {"response_1": chain_1, "question": itemgetter("question")} | RunnableLambda(route)

In [93]:
query = "Я ехал на велосипеде, упал и поцарапал машину, что делать и что мне будет?"
response = final_chain.invoke({"question": query})

Failed to use model_dump to serialize <class 'langchain_core.runnables.base.RunnableSequence'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_core.runnables.base.RunnableLambda'>)


In [94]:
Markdown(response.text_response)

Если вы упали на велосипеде и поцарапали машину, ваши действия и последствия будут зависеть от нескольких факторов, включая обстоятельства происшествия, наличие свидетелей и степень ущерба.

### Анализ запроса и контекста:

Ваш случай относится к области гражданского права, регулируемого Гражданским кодексом Российской Федерации (ГК РФ). Основные положения, которые применимы к вашему случаю, включают:

1. **Обязанность возмещения вреда** (ст. 1064 ГК РФ): Виновник причиненного вреда обязан его возместить. В вашем случае, если вы виновны в повреждении машины, вам придется возместить ущерб.

2. **Предположение вины** (п. 2 ст. 1064 ГК РФ): Вина причинителя вреда предполагается, если не доказано иное. Это означает, что если вы не сможете доказать, что не виноваты, вам придется возместить ущерб.

3. **Ответственность собственника транспортного средства** (ст. 1079 ГК РФ): В данном случае это положение не применимо, так как вы не являетесь собственником транспортного средства, которое было повреждено.

### Основные шаги, которые вам следует предпринять:

1. **Оценка ущерба**: Если владелец машины требует возмещения ущерба, вам может потребоваться проведение независимой оценки повреждений.

2. **Переговоры с владельцем машины**: Попробуйте договориться с владельцем машины о размере возмещения ущерба. Если ущерб незначительный, возможно, удастся решить вопрос мирным путем.

3. **Страхование**: Если у вас есть страховой полис, который покрывает такие случаи, свяжитесь со своей страховой компанией для получения консультации и возможного возмещения ущерба.

4. **Судебное разбирательство**: Если мирное урегулирование невозможно, владелец машины может подать на вас в суд. В этом случае вам следует подготовиться к судебному разбирательству, собрав все доказательства, подтверждающие вашу позицию.

### Рекомендации:

- **Сохраните все доказательства**: Фотографии места происшествия, записи с камер видеонаблюдения, показания свидетелей и т.д.

- **Консультация с юристом**: Если ситуация сложная, рекомендуется проконсультироваться с юристом, который поможет вам правильно оформить документы и подготовиться к судебному разбирательству.

- **Сообщите о происшествии в полицию**: Это может помочь в дальнейшем, если дело дойдет до суда.

### Ссылки на юридические документы:

- Гражданский кодекс Российской Федерации (ст. 1064, 1079)
- Федеральный закон от 25 апреля 2002 года №40-ФЗ «Об обязательном страховании гражданской ответственности владельцев транспортных средств»
- Постановление Пленума Верховного Суда РФ от 26.01.2010 N 1 «О применении судами гражданского законодательства, регулирующего отношения по обязательствам вследствие причинения вреда жизни или здоровью гражданина»
- Постановление Пленума Верховного Суда РФ от 23.06.2015 N 25 «О применении судами некоторых положений раздела I части первой Гражданского кодекса Российской Федерации»

In [95]:
response.legal_docs

['Гражданский кодекс Российской Федерации (ст. 1064, 1079)',
 'Федеральный закон от 25 апреля 2002 года №40-ФЗ «Об обязательном страховании гражданской ответственности владельцев транспортных средств»',
 'Постановление Пленума Верховного Суда РФ от 26.01.2010 N 1 «О применении судами гражданского законодательства, регулирующего отношения по обязательствам вследствие причинения вреда жизни или здоровью гражданина»',
 'Постановление Пленума Верховного Суда РФ от 23.06.2015 N 25 «О применении судами некоторых положений раздела I части первой Гражданского кодекса Российской Федерации»']